In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
pd.set_option('display.max_columns', None)
import warnings

warnings.filterwarnings("ignore")


In [2]:
df1 = pd.read_csv('data/football_rec/player_stats.csv',sep = ';',encoding = 'latin-1')
df1

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Goals,Shots,SoT,SoT%,G/Sh,G/SoT,ShoDist,ShoFK,ShoPK,PKatt,PasTotCmp,PasTotAtt,PasTotCmp%,PasTotDist,PasTotPrgDist,PasShoCmp,PasShoAtt,PasShoCmp%,PasMedCmp,PasMedAtt,PasMedCmp%,PasLonCmp,PasLonAtt,PasLonCmp%,Assists,PasAss,Pas3rd,PPA,CrsPA,PasProg,PasAtt,PasLive,PasDead,PasFK,TB,PasPress,Sw,PasCrs,CK,CkIn,CkOut,CkStr,PasGround,PasLow,PasHigh,PaswLeft,PaswRight,PaswHead,TI,PaswOther,PasCmp,PasOff,PasOut,PasInt,PasBlocks,SCA,ScaPassLive,ScaPassDead,ScaDrib,ScaSh,ScaFld,ScaDef,GCA,GcaPassLive,GcaPassDead,GcaDrib,GcaSh,GcaFld,GcaDef,Tkl,TklWon,TklDef3rd,TklMid3rd,TklAtt3rd,TklDri,TklDriAtt,TklDri%,TklDriPast,Press,PresSucc,Press%,PresDef3rd,PresMid3rd,PresAtt3rd,Blocks,BlkSh,BlkShSv,BlkPass,Int,Tkl+Int,Clr,Err,Touches,TouDefPen,TouDef3rd,TouMid3rd,TouAtt3rd,TouAttPen,TouLive,DriSucc,DriAtt,DriSucc%,DriPast,DriMegs,Carries,CarTotDist,CarPrgDist,CarProg,Car3rd,CPA,CarMis,CarDis,RecTarg,Rec,Rec%,RecProg,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
0,1,Max Aarons,ENG,DF,Norwich City,Premier League,22.0,2000,34,32,2881,32.0,0.00,0.41,0.06,15.4,0.00,0.00,20.5,0.00,0.00,0.00,34.0,45.0,75.5,574.1,214.8,17.5,19.4,90.0,13.1,17.00,77.0,3.06,6.78,45.2,0.06,0.59,1.56,1.13,0.25,2.94,45.0,34.4,10.60,0.84,0.06,5.41,0.59,1.41,0.00,0.00,0.00,0.0,26.50,9.59,8.94,4.91,29.00,0.91,9.72,0.06,34.0,0.22,0.88,1.63,1.75,1.19,0.84,0.06,0.09,0.13,0.06,0.00,0.16,0.16,0.0,0.00,0.00,0.00,0.0,2.16,1.16,1.56,0.59,0.00,1.16,1.81,63.8,0.66,13.6,3.53,26.0,7.97,4.38,1.22,2.69,0.69,0.03,2.00,1.75,3.91,2.19,0.00,58.0,5.06,23.30,23.8,15.00,0.91,47.8,1.03,2.44,42.3,1.09,0.19,33.9,199.4,121.7,5.44,1.66,0.41,0.84,0.94,36.0,32.4,89.9,1.28,0.25,0.00,0.00,0.97,1.84,0.03,1.41,1.16,0.00,0.06,0.03,5.53,0.47,1.59,22.7
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34.0,1987,34,34,2983,33.1,0.06,0.54,0.18,33.3,0.11,0.33,18.7,0.00,0.00,0.00,38.7,47.0,82.4,835.8,287.9,10.2,11.4,89.9,22.4,25.00,89.4,5.65,9.15,61.7,0.00,0.24,2.45,0.18,0.00,2.72,47.0,44.0,3.02,2.45,0.00,5.68,1.66,0.06,0.00,0.00,0.00,0.0,35.30,3.78,7.95,31.70,12.10,1.48,0.42,0.12,38.7,0.15,0.97,1.24,0.88,0.63,0.42,0.00,0.09,0.03,0.00,0.09,0.03,0.00,0.0,0.03,0.00,0.00,0.0,1.87,1.39,1.24,0.60,0.03,0.39,0.82,48.1,0.42,13.6,4.89,35.9,7.61,5.14,0.88,1.87,0.79,0.06,1.09,3.11,4.98,3.20,0.00,57.3,8.28,32.80,25.7,2.90,0.85,54.5,0.48,0.66,72.7,0.48,0.03,35.7,204.7,115.5,2.75,0.73,0.00,0.45,0.39,37.5,36.3,96.9,0.36,0.15,0.03,0.00,1.30,0.73,0.00,0.06,1.39,0.00,0.03,0.00,6.77,2.02,1.36,59.8
2,3,Salis Abdul Samed,GHA,MF,Clermont Foot,Ligue 1,22.0,2000,31,29,2462,27.4,0.04,0.66,0.18,27.8,0.06,0.20,20.3,0.00,0.00,0.00,55.9,61.0,91.7,1033.3,184.4,22.5,24.1,93.5,25.8,27.20,94.9,6.72,7.81,86.0,0.00,0.55,2.81,0.47,0.04,2.96,61.0,60.3,0.73,0.58,0.04,8.03,0.80,0.36,0.00,0.00,0.00,0.0,52.60,4.71,3.72,4.82,53.10,1.90,0.15,0.29,55.9,0.07,0.58,1.24,0.84,1.46,1.09,0.00,0.00,0.15,0.15,0.07,0.04,0.04,0.0,0.00,0.00,0.00,0.0,2.01,1.24,0.91,0.91,0.18,0.69,2.15,32.2,1.46,23.4,6.53,27.9,7.19,12.30,3.94,0.99,0.04,0.00,0.95,1.86,3.87,0.55,0.00,70.4,2.01,22.70,41.8,10.90,0.62,69.9,0.99,1.53,64.3,1.09,0.07,53.5,246.5,106.3,2.85,0.73,0.15,0.84,1.46,58.6,54.2,92.5,1.72,0.44,0.11,0.07,1.64,1.28,0.00,0.36,1.24,0.00,0.00,0.00,8.76,0.88,0.88,50.0
3,4,Laurent Abergel,FRA,MF,Lorient,Ligue 1,29.0,1993,34,34,2956,32.8,0.00,0.91,0.21,23.3,0.00,0.00,22.6,0.00,0.00,0.00,40.7,49.8,81.6,780.8,206.0,16.3,18.4,88.6,17.3,19.60,87.9,6.25,9.39,66.6,0.06,0.91,3.87,0.58,0.18,4.18,49.8,49.0,0.85,0.64,0.18,9.48,1.49,0.79,0.03,0.00,0.00,0.0,37.60,5.64,6.65,4.48,43.90,0.73,0.15,0.15,40.7,0.21,0.55,1.83,1.68,2.01,1.49,0.06,0.03,0.03,0.21,0.18,0.15,0.12,0.0,0.00,0.00,0.03,0.0,3.57,2.23,1.49,1.71,0.37,1.80,4.97,36.2,3.17,28.0,7.90,28.2,9.27,15.30,3.41,1.68,0.09,0.00,1.59,2.56,6.13,0.34,0.00,61.6,0.67,13.70,40.3,11.60,0.46,60.9,1.28,1.98,64.6,1.34,0.09,45.7,171.9,86.4,2.87,1.13,0.09,0.85,1.46,46.3,43.0,93.0,1.86,0.27,0.00,0.00,1.40,2.07,0.03,0.79,2.23,0.00,0.00,0.00,8.87,0.43,0.43,50.0
4,5,Charles A

In [3]:
df2 = pd.read_csv('../FIFA23_official_data.csv')
df2

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,Release Clause,Kit Number,Best Overall Rating
0,209658,L. Goretzka,27,https://cdn.sofifa.net/players/209/658/23_60.png,Germany,https://cdn.sofifa.net/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.net/teams/21/30.png,€91M,€115K,2312,Right,4.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos28"">SUB","Jul 1, 2018",NaN,2026,189cm,82kg,€157M,8.0,NaN
1,212198,Bruno Fernandes,27,https://cdn.sofifa.net/players/212/198/23_60.png,Portugal,https://cdn.sofifa.net/flags/pt.png,86,87,Manchester United,https://cdn.sofifa.net/teams/11/30.png,€78.5M,€190K,2305,Right,3.0,3.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos15"">LCM","Jan 30, 2020",NaN,2026,179cm,69kg,€155M,8.0,NaN
2,224334,M. Acuña,30,https://cdn.sofifa.net/players/224/334/23_60.png,Argentina,https://cdn.sofifa.net/flags/ar.png,85,85,Sevilla FC,https://cdn.sofifa.net/teams/481/30.png,€46.5M,€46K,2303,Left,2.0,3.0,3.0,High/ High,Stocky (170-185),No,"<span class=""pos pos7"">LB","Sep 14, 2020",NaN,2024,172cm,69kg,€97.7M,19.0,NaN
3,192985,K. De Bruyne,31,https://cdn.sofifa.net/players/192/985/23_60.png,Belgium,https://cdn.sofifa.net/flags/be.png,91,91,Manchester City,https://cdn.sofifa.net/teams/10/30.png,€107.5M,€350K,2303,Right,4.0,5.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos13"">RCM","Aug 30, 2015",NaN,2025,181cm,70kg,€198.9M,17.0,NaN
4,224232,N. Barella,25,https://cdn.sofifa.net/players/224/232/23_60.png,Italy,https://cdn.sofifa.net/flags/it.png,86,89,Inter,https://cdn.sofifa.net/teams/44/30.png,€89.5M,€110K,2296,Right,3.0,3.0,3.0,High/ High,Normal (170-),Yes,"<span class=""pos pos13"">RCM","Sep 1, 2020",NaN,2026,172cm,68kg,€154.4M,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17655,269526,Deng Xiongtao,19,https://cdn.sofifa.net/players/269/526/23_60.png,China PR,https://cdn.sofifa.net/flags/cn.png,48,61,Meizhou Hakka,https://cdn.sofifa.net/teams/114628/30.png,€100K,€500,762,Right,1.0,3.0,1.0,Medium/ Medium,Normal (185+),No,"<span class=""pos pos29"">RES","Apr 11, 2022",NaN,2027,190cm,78kg,€218K,35.0,NaN
17656,267946,22 Lim Jun Sub,17,https://cdn.sofifa.net/players/267/946/22_60.png,Korea Republic,https://cdn.sofifa.net/flags/kr.png,48,64,Jeju United FC,https://cdn.sofifa.net/teams/1478/30.png,€100K,€500,761,Right,1.0,2.0,1.0,Medium/ Medium,Lean (185+),No,"<span class=""pos pos29"">RES","Jan 1, 2022",NaN,2026,195cm,84kg,€188K,21.0,NaN
17657,270567,A. Demir,25,https://cdn.sofifa.net/players/270/567/23_60.png,Turkey,https://cdn.sofifa.net/flags/tr.png,51,56,Ümraniyespor,https://cdn.sofifa.net/teams/113796/30.png,€70K,€2K,759,Right,1.0,2.0,1.0,Medium/ Medium,Lean (185+),No,"<span class=""pos pos29"">RES","Jun 6, 2021",NaN,2023,190cm,82kg,€142K,12.0,NaN
17658,256624,21 S. Czajor,18,https://cdn.sofifa.net/players/256/624/21_60.png,Poland,https://cdn.sofifa.net/flags/pl.png,50,65,Fleetwood Town,https://cdn.sofifa.net/teams/112260/30.png,€90K,€500,758,Right,1.0,2.0,1.0,Medium/ Medium,Normal (185+),No,"<span class=""pos pos29"">RES","Jan 1, 2020",NaN,2021,187cm,79kg,€214K,40.0,NaN


In [4]:
df2.rename(columns={'Name':'Player','Club':'Squad','Nationality':'Nation'},inplace=True)

In [5]:
df2.columns

Index(['ID', 'Player', 'Age', 'Photo', 'Nation', 'Flag', 'Overall',
       'Potential', 'Squad', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight',
       'Release Clause', 'Kit Number', 'Best Overall Rating'],
      dtype='object')

In [6]:
df1.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP',
       'Starts',
       ...
       'Off', 'Crs', 'TklW', 'PKwon', 'PKcon', 'OG', 'Recov', 'AerWon',
       'AerLost', 'AerWon%'],
      dtype='object', length=143)

In [7]:
FIFA_COUNTRY_CODES = {
    'ENG': 'England',
    'MAR': 'Morocco',
    'GHA': 'Ghana',
    'FRA': 'France',
    'NGA': 'Nigeria',
    'PER': 'Peru',
    'ITA': 'Italy',
    'GER': 'Germany',
    'ARG': 'Argentina',
    'SCO': 'Scotland',
    'USA': 'United States',
    'ARM': 'Armenia',
    'SUI': 'Switzerland',
    'ESP': 'Spain',
    'COL': 'Colombia',
    'BIH': 'Bosnia and Herzegovina',
    'CIV': "Côte d'Ivoire",
    'NOR': 'Norway',
    'REU': 'Réunion',
    'EQG': 'Equatorial Guinea',
    'NED': 'Netherlands',
    'AUT': 'Austria',
    'CHI': 'Chile',
    'GUF': 'French Guiana',
    'PAR': 'Paraguay',
    'TUR': 'Turkey',
    'ROU': 'Romania',
    'BRA': 'Brazil',
    'GAB': 'Gabon',
    'SEN': 'Senegal',
    'WAL': 'Wales',
    'DEN': 'Denmark',
    'SWE': 'Sweden',
    'PAN': 'Panama',
    'JAM': 'Jamaica',
    'URU': 'Uruguay',
    'MEX': 'Mexico',
    'JPN': 'Japan',
    'KVX': 'Kosovo',
    'ALB': 'Albania',
    'TOG': 'Togo',
    'ALG': 'Algeria',
    'IRN': 'Iran',
    'CRO': 'Croatia',
    'LUX': 'Luxembourg',
    'CMR': 'Cameroon',
    'COD': 'DR Congo',
    'GRN': 'Grenada',
    'CZE': 'Czech Republic',
    'MKD': 'North Macedonia',
    'GAM': 'Gambia',
    'GUI': 'Guinea',
    'CPV': 'Cape Verde',
    'SUR': 'Suriname',
    'POL': 'Poland',
    'SVN': 'Slovenia',
    'SVK': 'Slovakia',
    'BEL': 'Belgium',
    'ISL': 'Iceland',
    'GNB': 'Guinea-Bissau',
    'MLI': 'Mali',
    'SRB': 'Serbia',
    'TUN': 'Tunisia',
    'NZL': 'New Zealand',
    'ECU': 'Ecuador',
    'POR': 'Portugal',
    'NIR': 'Northern Ireland',
    'RUS': 'Russia',
    'IRL': 'Republic of Ireland',
    'VEN': 'Venezuela',
    'ANG': 'Angola',
    'ISR': 'Israel',
    'ZAM': 'Zambia',
    'CAN': 'Canada',
    'MTQ': 'Martinique',
    'KOR': 'South Korea',
    'GRE': 'Greece',
    'BEN': 'Benin',
    'CRC': 'Costa Rica',
    'HON': 'Honduras',
    'EGY': 'Egypt',
    'MAD': 'Madagascar',
    'FIN': 'Finland',
    'CGO': 'Republic of the Congo',
    'DOM': 'Dominican Republic',
    'HUN': 'Hungary',
    'PHI': 'Philippines',
    'BUL': 'Bulgaria',
    'AUS': 'Australia',
    'UKR': 'Ukraine',
    'MNE': 'Montenegro',
    'BFA': 'Burkina Faso',
    'ZIM': 'Zimbabwe',
    'SLE': 'Sierra Leone',
    'CYP': 'Cyprus',
    'CTA': 'Central African Republic',
    'CHN': 'China',
    'GEO': 'Georgia',
    'MOZ': 'Mozambique',
    'RSA': 'South Africa',
    'CHA': 'Chad',
    'UZB': 'Uzbekistan',
}

In [8]:
df1['Nation'] = df1['Nation'].replace(FIFA_COUNTRY_CODES)

In [9]:
df1["Player"].fillna("", inplace=True)
df1["Squad"].fillna("", inplace=True)
df1["Nation"].fillna("", inplace=True)

df2["Player"].fillna("", inplace=True)
df2["Squad"].fillna("", inplace=True)
df2["Nation"].fillna("", inplace=True)

In [10]:
def preprocess_name(name):
    # Remove non-alphanumeric characters and convert to lowercase
    return re.sub(r'\W+', '', name).lower()

In [11]:
def most_similar_names(df1, df2, left_column, right_column, club_column, nation_column, name_weight=0.8, club_weight=0.2):
    result = pd.DataFrame(columns=[f'{left_column}_1', f'{right_column}_2', 'similarity'])

    # Iterate through unique nationalities
    for nation in df1[nation_column].unique():
        # Filter players with the same nationality in both dataframes
        df1_nation = df1[df1[nation_column] == nation]
        df2_nation = df2[df2[nation_column] == nation]

        if not df1_nation.shape[0] or not df2_nation.shape[0]:
            continue

        # Create separate combined data for names and clubs
        combined_names = [preprocess_name(name) for name in df1_nation[left_column].tolist()] + \
                         [preprocess_name(name) for name in df2_nation[right_column].tolist()]
        combined_clubs = [preprocess_name(club) for club in df1_nation[club_column].tolist()] + \
                         [preprocess_name(club) for club in df2_nation[club_column].tolist()]

        # Create a TfidfVectorizer object
        vectorizer_name = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
        vectorizer_club = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

        # Convert the combined data to separate matrices of TF-IDF features for names and clubs
        tfidf_matrix_name = vectorizer_name.fit_transform(combined_names)
        tfidf_matrix_club = vectorizer_club.fit_transform(combined_clubs)

        # Calculate the cosine similarity between names and clubs in df1_nation and df2_nation
        similarities_name = cosine_similarity(tfidf_matrix_name[:len(df1_nation)], tfidf_matrix_name[len(df1_nation):])
        similarities_club = cosine_similarity(tfidf_matrix_club[:len(df1_nation)], tfidf_matrix_club[len(df1_nation):])

        # Combine the similarities using the specified weights
        similarities = name_weight * similarities_name + club_weight * similarities_club

        # Find the most similar names' indices
        most_similar_indices = similarities.argmax(axis=1)

        # Append the most similar names to the result dataframe
        for index, row in df1_nation.iterrows():
            result = result.append({f'{left_column}_1': row[left_column],
                                    f'{right_column}_2': df2_nation.iloc[most_similar_indices[np.where(df1_nation.index == index)[0][0]]][right_column],
                                    'similarity': similarities[np.where(df1_nation.index == index)[0][0], most_similar_indices[np.where(df1_nation.index == index)[0][0]]]},
                                   ignore_index=True)

    return result

In [12]:
similar_names_df = most_similar_names(df1, df2, 'Player', 'Player', 'Squad', 'Nation')

In [13]:
similar_names_df.sort_values(by='similarity', ascending=True).head(50)

,Player_1,Player_2,similarity
2469,Sheraldo Becker,17 N. Hasselbaink,0.007029
2470,Ridgeciano Haps,17 N. Hasselbaink,0.018990
2397,Issa Bah,L. Barreiro Martins,0.032306
2327,Myrto Uzuni,20 G. Vrioni,0.038228
2754,Suf Podgoreanu,G. Assulin,0.047389
1846,Jesús Owono,Meseguer,0.048759
2329,Lorenz Assignon,D. Henen,0.050943
2646,Yoann Touzghar,Y. Benalouane,0.051037
2460,Ilaix Moriba,I. Sylla,0.051094
2458,Sehrou Guirassy,D. Bangoura,0.053884


In [14]:
temp = similar_names_df[similar_names_df['similarity'] > 0.35]

In [15]:
temp_with_id = pd.merge(temp, df2[['Player','ID']], left_on='Player_2', right_on='Player', how='left')
temp_with_id = temp_with_id[['Player_1', 'ID']]
temp_with_id.drop_duplicates(inplace=True)

In [16]:
final_df = pd.merge(df1, temp_with_id[['Player_1','ID']], left_on='Player', right_on='Player_1', how='left').drop(columns=['Player_1'])

In [17]:
final_df.fillna(0, inplace=True)

In [18]:
final_df = final_df.astype({'ID': 'int64'}).astype({'ID':'str'})#.to_csv('final.csv', index=False)

In [19]:
group_columns = [col if col != 'ID' else '' for col in final_df.columns]
# Group by all columns except 'ID' and choose the first occurrence of 'ID'
result = final_df.groupby(group_columns[:-1], as_index=False).agg({'ID': 'first'})

In [20]:
result.to_csv('data/football_rec/player_stats_processed.csv', index=False)

In [21]:
result

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%,ID
0,1,Max Aarons,England,DF,Norwich City,Premier League,22.0,2000,34,32,...,1.41,1.16,0.00,0.06,0.03,5.53,0.47,1.59,22.7,240060
1,2,Yunis Abdelhamid,Morocco,DF,Reims,Ligue 1,34.0,1987,34,34,...,0.06,1.39,0.00,0.03,0.00,6.77,2.02,1.36,59.8,206493
2,3,Salis Abdul Samed,Ghana,MF,Clermont Foot,Ligue 1,22.0,2000,31,29,...,0.36,1.24,0.00,0.00,0.00,8.76,0.88,0.88,50.0,252059
3,4,Laurent Abergel,France,MF,Lorient,Ligue 1,29.0,1993,34,34,...,0.79,2.23,0.00,0.00,0.00,8.87,0.43,0.43,50.0,213367
4,5,Charles Abi,France,FW,Saint-Étienne,Ligue 1,22.0,2000,1,1,...,2.00,0.00,0.00,0.00,0.00,4.00,2.00,0.00,100.0,243632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,2917,Martín Zubimendi,Spain,MF,Real Sociedad,La Liga,23.0,1999,36,26,...,0.10,1.08,0.00,0.00,0.00,9.41,2.64,1.42,65.0,248148
2917,2918,Szymon ?urkowski,Poland,MF,Empoli,Serie A,24.0,1997,35,29,...,0.74,1.91,0.00,0.04,0.00,8.05,1.02,1.56,39.4,239732
2918,2919,Martin Ødegaard,Norway,MF,Arsenal,Premier League,23.0,1998,36,32,...,0.84,0.81,0.00,0.06,0.00,7.44,0.55,0.81,40.5,222665
2919,2920,Milan ?uri?,Bosnia and Herzegovina,FW,Salernitana,Serie A,32.0,1990,33,23,...,0.29,0.17,0.04,0.00,0.00,3.24,12.40,4.81,72.0,0


In [32]:
df1[df1["Player"]=="Cristiano Ronaldo"].head(2)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Goals,Shots,SoT,SoT%,G/Sh,G/SoT,ShoDist,ShoFK,ShoPK,PKatt,PasTotCmp,PasTotAtt,PasTotCmp%,PasTotDist,PasTotPrgDist,PasShoCmp,PasShoAtt,PasShoCmp%,PasMedCmp,PasMedAtt,PasMedCmp%,PasLonCmp,PasLonAtt,PasLonCmp%,Assists,PasAss,Pas3rd,PPA,CrsPA,PasProg,PasAtt,PasLive,PasDead,PasFK,TB,PasPress,Sw,PasCrs,CK,CkIn,CkOut,CkStr,PasGround,PasLow,PasHigh,PaswLeft,PaswRight,PaswHead,TI,PaswOther,PasCmp,PasOff,PasOut,PasInt,PasBlocks,SCA,ScaPassLive,ScaPassDead,ScaDrib,ScaSh,ScaFld,ScaDef,GCA,GcaPassLive,GcaPassDead,GcaDrib,GcaSh,GcaFld,GcaDef,Tkl,TklWon,TklDef3rd,TklMid3rd,TklAtt3rd,TklDri,TklDriAtt,TklDri%,TklDriPast,Press,PresSucc,Press%,PresDef3rd,PresMid3rd,PresAtt3rd,Blocks,BlkSh,BlkShSv,BlkPass,Int,Tkl+Int,Clr,Err,Touches,TouDefPen,TouDef3rd,TouMid3rd,TouAtt3rd,TouAttPen,TouLive,DriSucc,DriAtt,DriSucc%,DriPast,DriMegs,Carries,CarTotDist,CarPrgDist,CarProg,Car3rd,CPA,CarMis,CarDis,RecTarg,Rec,Rec%,RecProg,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
2307,2308,Cristiano Ronaldo,Portugal,FW,Manchester Utd,Premier League,37.0,1985,30,27,2456,27.3,0.66,3.96,1.43,36.1,0.14,0.38,15.0,0.29,0.11,0.11,26.3,32.5,80.9,366.1,53.0,16.2,18.8,86.2,7.95,9.71,81.9,1.03,1.5,68.3,0.11,0.88,1.36,0.59,0.0,1.79,32.5,31.4,1.14,0.07,0.07,6.41,0.33,0.44,0.0,0.0,0.0,0.0,25.2,4.58,2.78,4.32,25.0,1.47,0.22,0.73,26.3,0.11,0.37,1.39,1.1,2.78,2.05,0.0,0.33,0.18,0.18,0.04,0.33,0.15,0.0,0.07,0.04,0.07,0.0,0.37,0.29,0.07,0.18,0.11,0.15,0.26,57.1,0.11,6.67,1.79,26.9,0.84,2.82,3.0,0.44,0.07,0.0,0.37,0.18,0.55,1.14,0.04,43.4,1.17,3.63,20.1,21.6,6.67,41.9,0.7,1.32,52.8,0.77,0.0,28.0,125.8,75.2,5.16,1.25,1.28,1.61,1.43,55.4,35.3,63.8,7.99,0.29,0.0,0.0,0.59,0.95,1.25,0.44,0.29,0.07,0.0,0.0,3.70,1.76,2.12,45.3
2308,2309,Cristiano Ronaldo,Portugal,FW,Juventus,Serie A,37.0,1985,1,0,31,0.3,0.00,3.33,0.00,0.0,0.00,0.00,9.0,0.00,0.00,0.00,30.0,40.0,75.0,380.0,46.7,26.7,33.3,80.0,3.33,6.67,50.0,0.00,0.0,0.0,0.00,3.33,0.00,0.00,0.0,0.00,40.0,36.7,3.33,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,30.0,6.67,3.33,3.33,33.3,0.00,0.00,3.33,30.0,0.00,0.00,3.33,0.0,6.67,3.33,0.0,0.00,0.00,3.33,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,3.33,3.33,0.00,0.00,56.7,0.00,6.67,30.0,26.7,6.67,53.3,0.0,0.00,0.0,0.00,0.0,40.0,243.3,110.0,6.67,6.67,0.00,0.00,6.67,46.7,40.0,85.7,10.00,3.33,0.0,0.0,0.00,3.33,3.33,0.00,0.00,0.00,0.0,0.0,6.67,0.00,0.00,0.0


In [41]:
df2[df2["ID"]==20801].head(2)

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,Release Clause,Kit Number,Best Overall Rating
100,20801,Cristiano Ronaldo,37,https://cdn.sofifa.net/players/020/801/23_60.png,Portugal,https://cdn.sofifa.net/flags/pt.png,90,90,Manchester United,https://cdn.sofifa.net/teams/11/30.png,€41M,€220K,2159,Right,5.0,4.0,5.0,High/ Low,Unique,Yes,"<span class=""pos pos25"">ST","Aug 27, 2021",NaN,2023,187cm,83kg,€77.9M,7.0,NaN


In [50]:
!pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.7 MB/s eta 0:00:00a 0:00:01


In [10]:
df1['text'] = df1['Player'] + ' ' + df1['Nation']
df2['text'] = df2['Player'] + ' ' + df2['Nation']

In [11]:
from fuzzywuzzy import process

for player in df1.text.unique():
    best_match = process.extractOne(player, df2['text'].values)
    print(f"{player}: {best_match}")

Max Aarons England: ('M. Aarons England', 91)
Yunis Abdelhamid Morocco: ('Y. Abdelhamid Morocco', 90)
Salis Abdul Samed Ghana: ('S. Abdul Samed Ghana', 89)
Laurent Abergel France: ('N. Kanté France', 86)
Charles Abi France: ('D. Charles Northern Ireland', 86)
Dickson Abiama Nigeria: ('D. Abiama Nigeria', 88)
Matthis Abline France: ('M. Abline France', 88)
Tammy Abraham England: ('T. Abraham England', 89)
Luis Abram Peru: ('Ó. Pinto Peru', 86)
Francesco Acerbi Italy: ('F. Acerbi Italy', 87)
Ragnar Ache Germany: ('R. Ache Germany', 87)
Mohamed Achi France: ('M. Achi France', 87)
Marcos Acuña Argentina: ('M. Acuña Argentina', 88)
Che Adams Scotland: ('C. Adams Scotland', 91)
Tyler Adams United States: ('T. Adams United States', 90)
Sargis Adamyan Armenia: ('E. Babayan Armenia', 70)
Martin Adeline France: ('J. Martin France', 88)
Amine Adli France: ('Y. Adli France', 87)
Yacine Adli France: ('Y. Adli France', 87)
Michel Aebischer Switzerland: ('M. Aebischer Switzerland', 90)
Felix Afena-Gy

KeyboardInterrupt: 

In [13]:
from nltk import ngrams
from nltk.metrics import jaccard_distance

def jaccard_similarity(s1, s2, n=3):
    s1_ngrams = set(ngrams(s1, n))
    s2_ngrams = set(ngrams(s2, n))
    return 1 - jaccard_distance(s1_ngrams, s2_ngrams)

def best_jaccard_match(player, names, n=3):
    similarities = [jaccard_similarity(player, name, n) for name in names]
    best_match_index = similarities.index(max(similarities))
    return names[best_match_index], similarities[best_match_index]

for player in df1.text.unique():
    best_match, similarity = best_jaccard_match(player, df2['text'].values)
    print(f"{player}: {best_match} ({similarity})")

Max Aarons England: M. Aarons England (0.7222222222222222)
Yunis Abdelhamid Morocco: Y. Abdelhamid Morocco (0.7083333333333333)
Salis Abdul Samed Ghana: S. Abdul Samed Ghana (0.6956521739130435)
Laurent Abergel France: M. Tel France (0.29166666666666663)
Charles Abi France: C. Abi France (0.5)
Dickson Abiama Nigeria: D. Abiama Nigeria (0.5909090909090908)
Matthis Abline France: M. Abline France (0.5714285714285714)
Tammy Abraham England: T. Abraham England (0.6666666666666667)
Luis Abram Peru: Luis Perea Spain (0.2272727272727273)
Francesco Acerbi Italy: F. Acerbi Italy (0.5)
Ragnar Ache Germany: R. Ache Germany (0.5789473684210527)
Mohamed Achi France: M. Achi France (0.5263157894736843)
Marcos Acuña Argentina: M. Acuña Argentina (0.6363636363636364)
Che Adams Scotland: C. Adams Scotland (0.7222222222222222)
Tyler Adams United States: T. Adams United States (0.72)
Sargis Adamyan Armenia: E. Babayan Armenia (0.33333333333333337)
Martin Adeline France: M. Adeline France (0.6190476190476